In [4]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
%matplotlib inline
pd.set_option('display.max_columns', 500)


### Playing around with using different columns that are complete

In [5]:
df = pd.read_csv('../data/clean_training_60_cols_chase.csv', parse_dates=['timestamp'], index_col="id")  
test = pd.read_csv("../data/clean_test_chase.csv", parse_dates=['timestamp'], index_col="id")
macro = pd.read_csv('../data/macro_chase.csv')
macro['quarter'] = pd.PeriodIndex(macro['Unnamed: 0'], freq='Q').strftime('Q%q-%y')
df['quarter'] = pd.PeriodIndex(df['timestamp'], freq='Q').strftime('Q%q-%y')

df = pd.merge(df,macro[['quarter','nominal_index']], how="left", on="quarter").reset_index(drop=True).set_index(df.index)

IOError: File ../data/clean_training_60_cols.csv does not exist

In [6]:
df['kitch_to_life'] = df.kitch_sq / df.life_sq
df['life_to_full'] = df.life_sq / df.full_sq
df['bld_type'] = 'med_rise'
df.loc[df.max_floor <= 5,'bld_type'] = 'low_rise'
df.loc[df.max_floor >= 17,'bld_type'] = 'high_rise'
df['walk_up_penalty'] = 0
df.loc[(df.floor>4) & (df.max_floor < 6),'walk_up_penalty'] = 1 

NameError: name 'df' is not defined

In [ ]:
df['adj_price_doc'] = df.price_doc / df.nominal_index
df['log_price'] = np.log(df.price_doc)
df['price_doc'] = df.price_doc / 1000
df['price_full'] = df.price_doc / df.full_sq

### Multiple Linear Regression

In [ ]:
df.drop('quarter',axis=1,inplace=True)
df_obj = df.select_dtypes(include=['object']).copy()
df_num = df.select_dtypes(exclude=['object'])

In [ ]:
num_uniques = df.nunique()

In [ ]:
from sklearn import linear_model
from sklearn.model_selection import KFold, cross_val_score

ols = linear_model.LinearRegression()

dummies = pd.get_dummies(df_obj)
lm_data = pd.concat([df_num,dummies],axis=1)

In [ ]:
cols = ['timestamp','price_doc','nominal_index','adj_price_doc','price_full','log_price','price_full']
cv_scores = cross_val_score(ols, lm_data.drop(cols,axis=1), lm_data.price_doc, cv=10)

In [ ]:
print cv_scores

In [ ]:
from sklearn.pipeline import make_pipeline

pipe  =  make_pipeline(MinMaxScaler(), Ridge())
param_grid = {'ridge__alpha': [100,10,1,0.1,0.01,0.001,0.0001,0]}
grid =  GridSearchCV(pipe, param_grid, cv=5)
grid.fit(lm_data.drop(cols,axis=1), df.adj_price_doc)